<a href="https://colab.research.google.com/github/cfreeman22/D213/blob/main/Copy_of_d213_task2_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import tensorflow as tf

In [70]:
device_list = tf.test.gpu_device_name()

In [71]:
device_list

''

In [72]:
!pip install contractions
import warnings
import warnings
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [73]:
import os
import json
import gzip
import pandas as pd
import re, string, unicodedata                          # Import Regex, string and unicodedata.
import contractions       
from bs4 import BeautifulSoup  
import nlp_prepare as prep
import matplotlib.pyplot as plt                         
import seaborn as sns
import nltk                                             # Import Natural Language Tool-Kit.
from collections import Counter
#nltk.download('stopwords')                              # Download Stopwords.
nltk.download('all')
#nltk.download('wordnet')
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from nltk.corpus import stopwords                       # Import stopwords.
from nltk.tokenize import word_tokenize, sent_tokenize  # Import Tokenizer.
from nltk.stem.wordnet import WordNetLemmatizer         # Import Lemmatizer.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

In [74]:
#reading the data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
file_path = '/content/drive/My Drive/Software.json.gz'


## __DATA PRE PROCESSING__

In [ ]:
data = prep.extract_data_and_process(file_path)

The shape of the original dataframe is (459436, 12)
Unique products grouped by productID is 21663
Unique users grouped by reviewerID is 375147
************************************************************************
************************************************************************
Dropping columns for our analysis : reviewerName, vote, image, style


In [ ]:
data.head()

## __DATA VISUALIZATION__
 

### __Distribution of sentiments across all the reviews__

In [ ]:
prep.labeled_barplot(data, "sentiments", perc=True)

### __WORDCLOUD for Negative, Positive and Neutral reviews Reviews__

In [ ]:
all_sentiments =['negative', 'positive','neutral']

In [ ]:
for x in all_sentiments:
  prep.generate_wordcloud(data, x)

In [ ]:
pd.set_option('display.max_colwidth', None) # Display full dataframe information (Non-turncated Text column.)

data.head()  

## __Data Processing:__
- Remove html tags.
- Replace contractions in string. (e.g. replace I'm --> I am) and so on.\
- Removal of Special Characters and Punctuations.
- Conversion to lowercase.
- Remove numbers.
- Tokenization
- Remove Stopwords.
- Lemmatized data
- We have used NLTK library to tokenize words , remove stopwords and lemmatize the remaining words.

In [ ]:
#remove the html tags
#remove the html tags
def strip_html(text):
    soup = BeautifulSoup(text, 'html')                    
    return soup.get_text()

#expand the contractions
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

#remove the numericals present in the text
def remove_numbers(text):
  text = re.sub(r'\d+', '', text)
  return text

def clean_text(text):
    text = strip_html(text)
    text = replace_contractions(text)
    text = remove_numbers(text)
    return text
 

In [ ]:
#data cleaning
data['review'] = data['review'].apply(lambda x: clean_text(x)) #[nltk.word_tokenize(line) for line in x ]
data.head()

In [ ]:
#get the no of words in every text
data['word_count'] = [len(t.split()) for t in data.review]
data.head()


In [ ]:
data['word_count'].describe()


In [ ]:
sns.set(style="whitegrid")
sns.boxplot(x=data['word_count'])

In [ ]:
mask = (data['word_count'] < 150) & (data['word_count'] >=20)
short_reviews = data[mask]
print('No of Short reviews')
print(len(short_reviews))

mask = data['word_count'] >= 150
long_reviews = data[mask]
print('No of Long reviews')
print(len(long_reviews))

In [ ]:
short_reviews['review'] = short_reviews.apply(lambda row: nltk.word_tokenize(row['review']), axis=1) # Tokenization of data
short_reviews.head()

In [ ]:
stopwords = stopwords.words('english')

customlist = ['not', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn',
        "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',
        "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn',
        "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# Set custom stop-word's list as not, couldn't etc. words matter in Sentiment, so not removing them from original data.

stopwords = list(set(stopwords) - set(customlist))                              

In [ ]:
lemmatizer = WordNetLemmatizer()

#remove the non-ASCII characters
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

# convert all characters to lowercase
def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

# Remove the punctuations
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

# Remove the stop words
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords:
            new_words.append(word)
    return new_words

# lemmatize the words
def lemmatize_list(words):
    new_words = []
    for word in words:
      new_words.append(lemmatizer.lemmatize(word, pos='v'))
    return new_words

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    words = lemmatize_list(words)
    return ' '.join(words)

short_reviews['review'] = short_reviews.apply(lambda row: normalize(row['review']), axis=1)
short_reviews.head()

### __Categories of sentiment to use__

The focus will be on positive and negative reviews only. The neutral reviews will be dropped

In [ ]:
review_data = short_reviews.copy()

In [ ]:
review_data = review_data[(review_data['sentiments']=='negative') | (review_data['sentiments']== 'positive')]

In [ ]:
prep.labeled_barplot(review_data, "sentiments", perc=True)

In [ ]:
review_data.sentiments = review_data.sentiments.replace(to_replace=['negative', 'positive'], value=[0, 1])



In [ ]:
review_data.head()

In [ ]:
#word count

In [ ]:
review_data.iloc[3]

In [ ]:
#clean
#cleaned_data = review_data.to_csv('cleaned_amazon_review_')

In [ ]:
# counting unique wword for the vocabulary size

def counter_word(text_col):
  count = Counter()
  for text in text_col.values:
    for word in text.split():
      count[word] += 1
  return count


counter = counter_word(review_data.review)

In [ ]:
len(counter)

In [ ]:
counter.most_common(10)

In [ ]:
vocab_size = len(counter)

In [ ]:
review_data.shape[0]

### __SPLIT THE DATA INTO TRAIN AND TEST SETS__

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = review_data.review
y = review_data.sentiments

X_train , X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.3)

# tokenize


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
# Setting up the split after tokenization
train_sentences = X_train.to_numpy()
train_labels = y_train.to_numpy()
test_sentences = X_test.to_numpy()
test_labels = y_test.to_numpy()
 

In [ ]:
# 16 dimensions
embedding_dim = 16
# Max length of 120 words per review
max_length = 150
# Truncating after 150 
trunc_type = 'post'
# Padding after 150
padding_type = 'post'

In [ ]:
# Starting the tokenizer
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
# Fitting the tokenizer
tokenizer.fit_on_texts(train_sentences)
# Getting our word index
word_index = tokenizer.word_index

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Setting sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences)
# Setting padding
train_padded = pad_sequences(train_sequences, maxlen=max_length,padding=padding_type, truncating=trunc_type)
test_sentences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sentences, maxlen=max_length,padding=padding_type, truncating=trunc_type)

In [ ]:
train_padded[70]

In [ ]:
# Creating model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='relu')
])

In [ ]:
# Compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
import numpy as np

In [ ]:
# Training the model
training_final = np.array(train_labels)
testing_final = np.array(test_labels)

In [ ]:
# Running the model for 25 epochs
model1 = model.fit(train_padded, train_labels, epochs=25, validation_data=(test_padded,
test_labels))

In [ ]:
# Capturing learning history per epoch
hist  = pd.DataFrame(model1.history)
hist['epoch'] = model1.epoch

# Plotting accuracy at different epochs
plt.plot(hist['loss'])
plt.plot(hist['val_loss'])
plt.legend(("train" , "valid") , loc =0)

 